#### This notebook will scrape information on Toronto neighbourhoods from Wikipedia and then cluster them based on venue information from Foursquare

In [1]:
import pandas as pd
import numpy as np

#### Part 1

Scrape data from wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)

toronto_nbhds=tables[0].iloc[1:]
toronto_nbhds.head()

,Postal Code,Borough,Neighborhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


Drop rows where Borough is "Not assigned"

In [3]:
to_nbhds=toronto_nbhds.drop(toronto_nbhds[toronto_nbhds.Borough=="Not assigned"].index)
to_nbhds.sort_values(by=['Postal Code'])
to_nbhds.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Group rows by Postal Code and Borough, concatenating neighbourhoods into a comma-separated string

In [4]:
to_nbhds['Neighborhood'] = to_nbhds.groupby(['Postal Code','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
to_nbhds.sort_values(by='Borough')
to_nbhds.reset_index(drop=True,inplace=True)
to_nbhds.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
to_nbhds.shape

(103, 3)

#### Part 2

Import csv containing lat lon data for Toronto postcodes and join with to_nbhds dataset

In [6]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print("Data downloaded!")

Data downloaded!


In [7]:
geo_data=pd.read_csv("Geospatial_data.csv")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
tro_merged=to_nbhds.join(geo_data.set_index('Postal Code'), on='Postal Code')
tro_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
tro_merged.shape

(103, 5)

#### Part 3

Exploring and clustering neighborhoods based on venue data from Foursquare

In [10]:
# View unique boroughs
tro_merged['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [20]:
# Limit dataset to boroughs containing "Toronto"
tro_filtered=tro_merged[tro_merged['Borough'].str.contains("Toronto")]
tro_filtered['Borough'].unique()
tro_filtered.reset_index(drop=True,inplace=True)
tro_filtered.shape

(39, 5)

Import key packages for creating maps and accessing/parsing Foursquare data

In [14]:
# Install Folium if required
!pip install folium

     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011


In [15]:
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from geopy.geocoders import Nominatim

print('Libraries imported.')

Libraries imported.


Create map of Toronto

In [16]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(tro_filtered['Latitude'], tro_filtered['Longitude'], tro_filtered['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now we'll get data from the Foursquare API

Firstly, we set credentials

In [63]:
# The code was removed by Watson Studio for sharing.

Your credentials:
CLIENT_ID: 0M243W1YQQSMXJO2E1SJGDIZWS5FT1UBS5EHZ5MWVPUP4CNP
CLIENT_SECRET:FUMOBOWWO2CIYPSBGL20SNCWXF1MPMLHUFJX4FC2ZJMUXHGF


Next, let's identify the first Postal Code, and get the first 100 venues within a radius of 500 meters

In [21]:
pc_latitude = tro_filtered.loc[0, 'Latitude'] # Postal Code latitude value
pc_longitude = tro_filtered.loc[0, 'Longitude'] # Postal Code longitude value

pc_code = tro_filtered.loc[0, 'Postal Code'] # Postal Code

print('Latitude and longitude values of {} are {}, {}.'.format(pc_code, 
                                                               pc_latitude, 
                                                               pc_longitude))

Latitude and longitude values of M5A are 43.6542599, -79.3606359.


Now we create our url for the API query

In [22]:
limit = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    pc_latitude, 
    pc_longitude, 
    radius, 
    limit)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=0M243W1YQQSMXJO2E1SJGDIZWS5FT1UBS5EHZ5MWVPUP4CNP&client_secret=FUMOBOWWO2CIYPSBGL20SNCWXF1MPMLHUFJX4FC2ZJMUXHGF&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

And now we can run the query and review the results

In [62]:
results = requests.get(url).json()

We can create a function to get the category type of the venue, and then turn the results into a *pandas* data frame

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874


We can then replicate this for each of the Postal Code districts in our filtered list

In [30]:
# Define function to loop through the various districts and extract the venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=tro_filtered['Postal Code'],
                                 latitudes=tro_filtered['Latitude'],
                                 longitudes=tro_filtered['Longitude']
                                )

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


Lets check the size of the resulting data frame, along with a sample of the output

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(1614, 7)


,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [33]:
toronto_venues.groupby('Postal Code').count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,4,4,4,4,4,4
M4K,43,43,43,43,43,43
M4L,22,22,22,22,22,22
M4M,40,40,40,40,40,40
M4N,3,3,3,3,3,3
M4P,8,8,8,8,8,8
M4R,22,22,22,22,22,22
M4S,34,34,34,34,34,34
M4T,4,4,4,4,4,4


In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


Analyze each Postal District using one-hot encoding to work out proportion of venues falling into each category

In [38]:
# one hot encoding
tro_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tro_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [tro_onehot.columns[-1]] + list(tro_onehot.columns[:-1])
tro_onehot = tro_onehot[fixed_columns]

tro_onehot.head()

tro_grouped = tro_onehot.groupby('Postal Code').mean().reset_index()
tro_grouped

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.023256,0.000000,0.00,...,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256
2,M4L,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.050000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.025000
4,M4N,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
7,M4S,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.062500,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000


Identify the 5 most common venues in each postal district

In [40]:
num_top_venues = 5

for district in tro_grouped['Postal Code']:
    print("----"+district+"----")
    temp = tro_grouped[tro_grouped['Postal Code'] == district].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2              Trail  0.25
3       Neighborhood  0.25
4               Park  0.00


----M4K----
                    venue  freq
0        Greek Restaurant  0.19
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4          Ice Cream Shop  0.05


----M4L----
                  venue  freq
0                  Park  0.14
1        Sandwich Place  0.09
2  Fast Food Restaurant  0.09
3                   Pub  0.05
4            Steakhouse  0.05


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3  American Restaurant  0.05
4            Gastropub  0.05


----M4N----
               venue  freq
0               Park  0.33
1           Bus Line  0.33
2        Swim School  0.33
3  Afghan Restaurant  0.00
4             Museum  0.00


----M4P----
                  venue  freq
0           Pizza 

Let's transform this output into a *pandas* data frame

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['Postal Code'] = tro_grouped['Postal Code']

for ind in np.arange(tro_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tro_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio
1,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Restaurant
2,M4L,Park,Fast Food Restaurant,Sandwich Place,Brewery,Steakhouse
3,M4M,Café,Coffee Shop,Bakery,Gastropub,American Restaurant
4,M4N,Park,Bus Line,Swim School,Yoga Studio,Diner


And now lets cluster these districts based on their most common venues, and overlay these clusters onto a map

In [44]:
# set number of clusters
kclusters = 5

tro_grouped_clustering = tro_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tro_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 2, 3, 0, 3, 3, 2, 3, 2], dtype=int32)

In [47]:
# add clustering labels
districts_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tro_merged = tro_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tro_merged = tro_merged.join(districts_venues_sorted.set_index('Postal Code'), on='Postal Code')

tro_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Bakery,Pub,Park,Breakfast Spot
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Sushi Restaurant,Yoga Studio,College Cafeteria,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tro_merged['Latitude'], tro_merged['Longitude'], tro_merged['Postal Code'], tro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally, let's review each of the clusters

In [56]:
tro_merged.loc[tro_merged['Cluster Labels'] == 0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,0,Park,Bus Line,Swim School,Yoga Studio,Diner


In [57]:
tro_merged.loc[tro_merged['Cluster Labels'] == 1]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
21,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Jewelry Store,Trail,Park,Sushi Restaurant,Discount Store
33,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Department Store


In [58]:
tro_merged.loc[tro_merged['Cluster Labels'] == 2]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio
15,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Park,Fast Food Restaurant,Sandwich Place,Brewery,Steakhouse
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,2,Sandwich Place,Café,Coffee Shop,Pub,Donut Shop
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Thai Restaurant
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,2,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Pub
31,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,2,Coffee Shop,Pub,Liquor Store,Supermarket,Sushi Restaurant


In [59]:
tro_merged.loc[tro_merged['Cluster Labels'] == 3]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Bakery,Pub,Park,Breakfast Spot
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Sushi Restaurant,Yoga Studio,College Cafeteria,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Diner,Baby Store
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3,Coffee Shop,Café,Restaurant,Clothing Store,Hotel
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,3,Bakery,Pharmacy,Pet Store,Middle Eastern Restaurant,Music Venue
10,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,3,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint


In [60]:
tro_merged.loc[tro_merged['Cluster Labels'] == 4]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,M5N,Central Toronto,Roselawn,43.711695,-79.416936,4,Garden,Home Service,Ice Cream Shop,Music Venue,Ethiopian Restaurant
